## Data Analysis

### Importing libraries and uploading data

In [98]:
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
import scipy.stats as st

In [63]:
trips_amount = pd.read_csv('datasets\project_sql_result_01.csv')
# trips_amount: o número de corridas para cada empresa de táxi de 15 a 16 de novembro de 2017.

trips_details = pd.read_csv('datasets\project_sql_result_04.csv')
# dropoff_location_name: bairros de Chicago onde as corridas terminaram
# average_trips: o número médio de viagens que terminaram em cada bairro em novembro de 2017.

loop_airprot_travels = pd.read_csv('datasets\project_sql_result_07.csv')

### Checking for inconsistencies - trips_amount data


In [4]:
trips_amount.sample(3)

,company_name,trips_amount
55,6057 - 24657 Richard Addo,13
39,0118 - 42111 Godfrey S.Awir,33
46,3620 - 52292 David K. Cab Corp.,21


In [5]:
trips_amount.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   company_name  64 non-null     object
 1   trips_amount  64 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ KB


In [6]:
trips_amount.duplicated().sum()

0

### Checking for inconsistencies - trips_details data

In [7]:
trips_details.sample(3)
trips_details['rounded_average'] = trips_details['average_trips'].round(2)
trips_details.sample(3)

,dropoff_location_name,average_trips,rounded_average
91,Burnside,2.333333,2.33
83,Avalon Park,6.966667,6.97
60,New City,22.933333,22.93


In [8]:
trips_details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94 entries, 0 to 93
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   dropoff_location_name  94 non-null     object 
 1   average_trips          94 non-null     float64
 2   rounded_average        94 non-null     float64
dtypes: float64(2), object(1)
memory usage: 2.3+ KB


In [9]:
trips_details.duplicated().sum()

0

### Data Visualization - Trips_amount and trips_details datasets

In [10]:
### Identifying the 10 most popular locations
popular_locations = trips_details.sort_values(by = 'average_trips', ascending = False).head(10)
popular_locations

,dropoff_location_name,average_trips,rounded_average
0,Loop,10727.466667,10727.47
1,River North,9523.666667,9523.67
2,Streeterville,6664.666667,6664.67
3,West Loop,5163.666667,5163.67
4,O'Hare,2546.900000,2546.90
5,Lake View,2420.966667,2420.97
6,Grant Park,2068.533333,2068.53
7,Museum Campus,1510.000000,1510.00
8,Gold Coast,1364.233333,1364.23
9,Sheffield & DePaul,1259.766667,1259.77


In [49]:
# Taxi companies - bar graph
trips_amount_bar = px.bar(trips_amount,
                          x ='company_name',
                          y ='trips_amount',
                          title = 'Total trips amount for each taxi company')

# Updating x_axis and y_axis titles.
trips_amount_bar.update_layout(xaxis_title='Company',
                               yaxis_title='Amount')

#Updating the x_axes to show all the companies, instead of the default.
trips_amount_bar.update_xaxes(tickmode='linear',
                              dtick=1)

The outcome in the bar graph above is inneficient. Even though we are using plotly.express library that allows us to zoom in the graph, what we can infer is that there is a high discrepance between the company with the highest average amount of trips and the lowest one.  
  
Therefore a bar graph might not be ideal in this situation. Let's visualize this information through boxplot below:

In [50]:
#Total trips amount
trips_amount_box = px.box(trips_amount,
                          x ='trips_amount',
                          title = 'Total trips amount - Boxplot',
                          points = 'all')

# Updating x_axis and y_axis titles.
trips_amount_box.update_layout(xaxis_title='Amount')

trips_amount_box


There are relevant informations here, such as:  
- The lowest amount of trips run by a taxi company was 2;  
- The median is 178.5;
- The companies with an amount higher than 5015 are outliers.

In [51]:
#Average trips amount
trips_amount_hist = px.histogram(trips_amount,
                          x ='trips_amount',
                          title = 'Taxi companies performance analysis by their total amount of trips - Histogram',
                          nbins = 40)
trips_amount_hist.update_layout(xaxis_title = 'Average of trips')
trips_amount_hist


Above we have a histogram where we can determine the following:  
- Most taxi companies had a performance below 500 trips;
- Only 1 company had a performance close to 20000.

Let's check a bar graph using our median discovered in our boxpot.:

In [52]:
# Taxi Companies
trips_amount_bar_by_box = px.bar(trips_amount[trips_amount['trips_amount']<=178],
                          x ='company_name',
                          y ='trips_amount',
                          title = 'Total of trips for each taxi company below or equal to 178')

# Updating x_axis and y_axis titles.
trips_amount_bar_by_box.update_layout(xaxis_title='Company',
                               yaxis_title='Amount')

#Updating the x_axes to show all the companies, instead of the default.
trips_amount_bar_by_box.update_xaxes(tickmode='linear',
                              dtick=1)

In the graph above we can see the taxi companies that were inside of the Q1 and Q2 in our boxplot.

In [53]:
# Taxi Companies
trips_amount_bar_by_box_q3 = px.bar(trips_amount[(trips_amount['trips_amount']>178) & (trips_amount['trips_amount']<5015)],
                          x ='company_name',
                          y ='trips_amount',
                          title = 'Average of trips for each taxi company with total above 178 and less than 5015')

# Updating x_axis and y_axis titles.
trips_amount_bar_by_box_q3.update_layout(xaxis_title='Company',
                               yaxis_title='Amount')

#Updating the x_axes to show all the companies, instead of the default.
trips_amount_bar_by_box_q3.update_xaxes(tickmode='linear',
                             dtick=1,
                             tickangle=45)

Above we have a graph with the data of the companies that performed above the median and below our upper fence of 5015. Above there is the companies that performed above 5015 and are the outliers in our boxplot.

In [55]:
# Taxi Companies
trips_amount_bar_by_box_outliers = px.bar(trips_amount[trips_amount['trips_amount']>5015],
                          x ='company_name',
                          y ='trips_amount',
                          title = 'Average of trips for each taxi company with total above 5015 (Outliers)')

# Updating x_axis and y_axis titles.
trips_amount_bar_by_box_outliers.update_layout(xaxis_title='Company',
                               yaxis_title='Amount')

#Updating the x_axes to show all the companies, instead of the default.
trips_amount_bar_by_box_outliers.update_xaxes(tickmode='linear',
                             dtick=1,
                             tickangle=45)

In [62]:
popular_locations_bar = px.bar(popular_locations,
                               x = 'dropoff_location_name',
                               y = 'average_trips',
                               title = 'Most popular taxi drop-off locations')

popular_locations_bar.update_layout(xaxis_title='Drop-off location',
                               yaxis_title='Total')

popular_locations_bar

### Checking for inconsistencies - loop_airport_travels data

In [70]:
loop_airprot_travels.sample(3)

,start_ts,weather_conditions,duration_seconds
1011,2017-11-18 14:00:00,Good,3660.0
578,2017-11-11 14:00:00,Good,3614.0
92,2017-11-18 07:00:00,Bad,1511.0


In [71]:
loop_airprot_travels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1068 entries, 0 to 1067
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   start_ts            1068 non-null   object 
 1   weather_conditions  1068 non-null   object 
 2   duration_seconds    1068 non-null   float64
dtypes: float64(1), object(2)
memory usage: 25.2+ KB


In [74]:
loop_airprot_travels['start_ts'] = pd.to_datetime(loop_airprot_travels['start_ts'],format = '%Y-%m-%d %H:%M:%S')

In [80]:
loop_airprot_travels['weather_conditions'].unique()

array(['Good', 'Bad'], dtype=object)

In [84]:
loop_airprot_travels['week_day'] = loop_airprot_travels['start_ts'].dt.day_name()

In [85]:
loop_airprot_travels.sample()

,start_ts,weather_conditions,duration_seconds,week_day
531,2017-11-18 12:00:00,Bad,2220.0,Saturday


### Data Visualizations - loop_airport_travels_data

In [88]:
loop_airprot_travels.head(5)

,start_ts,weather_conditions,duration_seconds,week_day
0,2017-11-25 16:00:00,Good,2410.0,Saturday
1,2017-11-25 14:00:00,Good,1920.0,Saturday
2,2017-11-25 12:00:00,Good,1543.0,Saturday
3,2017-11-04 10:00:00,Good,2512.0,Saturday
4,2017-11-11 07:00:00,Good,1440.0,Saturday


In [102]:
bad_weather_mean = loop_airprot_travels[loop_airprot_travels['weather_conditions'] == 'Bad']['duration_seconds'].mean()
good_weather_mean = loop_airprot_travels[loop_airprot_travels['weather_conditions'] == 'Good']['duration_seconds'].mean()

print('Bad weather average duration in minutes: ',bad_weather/60)  
print('Good weather average duration in minutes: ',good_weather/60)

Bad weather average duration in minutes:  40.45342592592593
Good weather average duration in minutes:  33.327927927927924


In [122]:
# The average duration of trips during different types of weather are not differents

# Separating data
bad_weather = loop_airprot_travels[loop_airprot_travels['weather_conditions']=='Bad']['duration_seconds']
good_weather = loop_airprot_travels[loop_airprot_travels['weather_conditions']=='Good']['duration_seconds']


# Teste as hipóteses

alpha = 0.05

results = st.ttest_ind(bad_weather, good_weather,equal_var=False)

print('Valor-p:', results.pvalue,'\n')


if results.pvalue < alpha:
    print('We reject the null hypothesis.')
    print('The average duration of trips during bad weather is different from that during good weather.')
else:
    print('We fail to reject the null hypothesis.')
    print('There is no significant difference in the average duration of trips during bad and good weather.')

Valor-p: 6.517970327099473e-12 

We reject the null hypothesis.
The average duration of trips during bad weather is different from that during good weather.
